In [1]:
#IPython extension to reload modules before executing user code. If we make any change in function 
# and imoprting it we don't need to restart if we use auto reload 
%load_ext autoreload

# # to print in between values as well without print
# from IPython.core.interactiveshell import InteractiveShell
# InteractiveShell.ast_node_interactivity = "all"


#hide warnings
import warnings
warnings.filterwarnings('ignore')

In [2]:
# import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sn
import pickle

# sklearn libraries
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import LocalOutlierFactor
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder
from sklearn import preprocessing

In [3]:
# to save pickle file
def save_pkl_model(path,model):
    pickle.dump(model, open(path,'wb'))

In [4]:
# to load
def load_pkl_model(path):
    sc = pickle.load(open(path,'rb'))
    return sc

In [5]:
def preprocess_pipeline_1(test):
    # miss_num_imputer = load_pkl_model(missing_num_imputer_path)
    # missing_cat_imputer = load_pkl_model(missing_cat_imputer_path)
    ohe_converter = load_pkl_model(ohe_converter_path)
    num_transformer = load_pkl_model(mms_converter_path)

    # processing numeric
    num_cols = test.select_dtypes(include='number')
    num_column_names = num_cols.columns
    # num_miss_processed = miss_num_imputer.transform(num_cols)
    num_scalled = num_transformer.transform(num_cols)
    num_scalled_df = pd.DataFrame(num_scalled,columns=num_column_names)


    # processing categoric
    cat_cols = test.select_dtypes(include='O')
    # cat_miss_processed = missing_cat_imputer.transform(cat_cols)
    cat_encoded = ohe_converter.transform(cat_cols).toarray()
    cat_encoded_df = pd.DataFrame(cat_encoded,columns=ohe_converter.get_feature_names_out())


    # combine categoric and numeric data and return
    test_processed = pd.concat([num_scalled_df,cat_encoded_df],axis=1)

    return test_processed
    

In [6]:
# spacify saved preprocessing model paths
# missing_num_imputer_path = 'Data/processing_models/imputer_num.pkl'
# missing_cat_imputer_path = 'Data/processing_models/imputer_cat.pkl'
ohe_converter_path = 'Data/processing_models/ohe_converter.pkl'
mms_converter_path = 'Data/processing_models/mms_converter.pkl'

In [7]:
train_data = pd.read_csv('Data/transformed/train_mms_ohe.csv')
train_data.head()

,raisedhands,VisITedResources,AnnouncementsView,Discussion,gender_M,NationalITy_Iran,NationalITy_Iraq,NationalITy_Jordan,NationalITy_KW,NationalITy_Lybia,...,Topic_Math,Topic_Quran,Topic_Science,Topic_Spanish,Semester_S,Relation_Mum,ParentAnsweringSurvey_Yes,ParentschoolSatisfaction_Good,StudentAbsenceDays_Under-7,Class
0,0.50,0.888889,0.306122,0.806122,1.0,0.0,0.0,0.0,1.0,0.0,...,1.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,H
1,0.32,0.828283,0.602041,0.632653,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,1.0,1.0,0.0,0.0,M
2,0.50,0.626263,0.132653,0.326531,1.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,L
3,0.60,0.808081,0.510204,0.397959,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,M
4,0.70,0.929293,0.510204,0.061224,0.0,0.0,0.0,0.0,1.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,H


In [8]:
# load test data and check
test_data = pd.read_csv('Data/test.csv')
# test_data.head()
# devide Xtest ytest to compare
X_test,y_test_org = test_data.loc[:,~test_data.columns.isin(['Class'])],test_data['Class']
# preporcess the data for the model input
X_test_processed = preprocess_pipeline_1(X_test)
# X_test_processed.head()

In [9]:
# label_to_key = {}
# key_to_label = {}
# for i,label in enumerate(train_data['Class'].unique()):
#     label_to_key[label] = i
#     key_to_label[i]= label

In [10]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.optim import SGD


In [11]:
def ohe_converter(train):
    ohe = OneHotEncoder(handle_unknown = 'ignore') 
    encoded_vec = ohe.fit(train)
    ohe_transformed = encoded_vec.transform(train).toarray()
    return ohe_transformed,ohe

In [12]:
X_train = train_data.iloc[:,:-1].values
# y_train = train_data.iloc[:,-1].apply(lambda x: label_to_key[x]).values
# y_train = train_data.iloc[:,-1].values
y_train = train_data.iloc[:,-1].values
y_train = y_train.reshape(-1,1)
y_train,ohe_train = ohe_converter(y_train)

X_test = X_test_processed.values
# y_test = y_test.apply(lambda x: label_to_key[x]).values
y_test = y_test_org.values
y_test = y_test.reshape(-1,1)
y_test = ohe_train.transform(y_test).toarray()

In [13]:
X_train.shape

(408, 60)

In [14]:
X_test.shape

(72, 60)

In [15]:
X_train = torch.Tensor(X_train)
y_train = torch.Tensor(y_train)

In [16]:
X_test = torch.Tensor(X_test)
y_test = torch.Tensor(y_test)

In [20]:
# Pytorch train and test sets
train = torch.utils.data.TensorDataset(X_train,y_train)
test = torch.utils.data.TensorDataset(X_test,y_test)

In [21]:
# data loader
train_loader = torch.utils.data.DataLoader(train, batch_size = 5, shuffle = False)
test_loader = torch.utils.data.DataLoader(test, batch_size = 5, shuffle = False)

In [22]:
# next(iter(train_loader))

In [23]:
# Display image and label.
train_features, train_labels = next(iter(train_loader))
print(f"Feature batch shape: {train_features.size()}")
print(f"Labels batch shape: {train_labels.size()}")

Feature batch shape: torch.Size([5, 60])
Labels batch shape: torch.Size([5, 3])


In [24]:
# train_features.view(1,5,60).shape

In [25]:
# Create ANN Model
# we create a new class ANNModel which inherits from nn.Module 
# define variables in a class and a forward function which uses those variables and perform forward pass 
class ANNModel(nn.Module):
    
    def __init__(self, input_dim, hidden_dim, output_dim):
        super(ANNModel, self).__init__()
        
        # Linear function 1: 60 --> 32
        self.fc1 = nn.Linear(input_dim, hidden_dim) 
        # Non-linearity 1
        # self.relu1 = nn.ReLU()
        
        # Linear function 2: 32 --> 32
        self.fc2 = nn.Linear(hidden_dim, hidden_dim)
        # Non-linearity 2
        # self.tanh2 = nn.Tanh()
        
        # Linear function 3: 32 --> 32
        self.fc3 = nn.Linear(hidden_dim, hidden_dim)
        # Non-linearity 3
        # self.elu3 = nn.ELU()
        
        # Linear function 4 (readout): 32 --> 1
        self.fc4 = nn.Linear(hidden_dim, output_dim)  
    
    def forward(self, x):
        # Linear function 1
        out = F.relu(self.fc1(x))
        # Non-linearity 1
        # out = self.relu1(out)
        
        # Linear function 2
        out = F.relu(self.fc2(out)) 
        # Non-linearity 2
        # out = self.relu1(out)
        
        # Linear function 2
        out = F.relu(self.fc3(out))
        # Non-linearity 2
        # out = self.relu1(out)
        
        # Linear function 4 (readout)
        out = self.fc4(out)
        return out
        # return F.softmax(out,dim=1)

In [26]:
# instantiate ANN
input_dim = train_features.size()[1]
hidden_dim = 32 #hidden layer dim is one of the hyper parameter and it should be chosen and tuned. For now I only say 150 there is no reason.
output_dim = 3

# Create ANN
model = ANNModel(input_dim, hidden_dim, output_dim)
# to check all working fine, we can simply call model, no need to call specificly forward
# model(train_features)
# model.forward(train_features)

In [27]:
# Cross Entropy Loss 
error = nn.CrossEntropyLoss()

# SGD Optimizer
# model parameters are those variables for which gradient calculation is true, which can be updated
learning_rate = 0.02
optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)

In [28]:
model.parameters

<bound method Module.parameters of ANNModel(
  (fc1): Linear(in_features=60, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=32, bias=True)
  (fc3): Linear(in_features=32, out_features=32, bias=True)
  (fc4): Linear(in_features=32, out_features=3, bias=True)
)>

In [30]:
EPOCHS = 100

for epoch in range(EPOCHS):
    for data in train_loader:
        # get the batch
        X,y = data
        # make the gradients zero so get ready for next pass else keep getting add
        model.zero_grad()
        output = model(X)

        # depends if our data is one hot encoded 
        # print(output)
        print(y)
        print(output)

        loss =error(output,y)

        # calculate gradient
        loss.backward()

        # update gradient
        optimizer.step()
        print(loss)        

tensor([[1., 0., 0.],
        [0., 0., 1.],
        [0., 1., 0.],
        [0., 0., 1.],
        [1., 0., 0.]])
tensor([[  7.9498, -12.8731,   4.6099],
        [ -3.8899,   0.8368,   3.9021],
        [-13.8551,  10.7241,   5.2739],
        [  4.4444, -12.3678,   8.2801],
        [  9.6018, -15.2109,   5.4411]], grad_fn=<AddmmBackward0>)
tensor(0.0244, grad_fn=<DivBackward1>)
tensor([[0., 1., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [1., 0., 0.],
        [0., 1., 0.]])
tensor([[ -8.1687,   6.7640,   2.6842],
        [ 15.5315, -25.8589,   9.6434],
        [ 15.2644, -25.4111,   9.3758],
        [  9.8850, -16.5092,   6.2734],
        [-12.7126,   9.9398,   4.7483]], grad_fn=<AddmmBackward0>)
tensor(0.0109, grad_fn=<DivBackward1>)
tensor([[0., 1., 0.],
        [0., 0., 1.],
        [1., 0., 0.],
        [0., 1., 0.],
        [1., 0., 0.]])
tensor([[-21.6717,  16.2511,   8.8013],
        [ -1.2545,  -6.4589,   8.7338],
        [  4.9876,  -8.0898,   2.8917],
        [-12.27

In [44]:
correct = 0
total = 0
results = []

with torch.no_grad():
    for t_data in test_loader:
        X_t,y_t = t_data
        test_output = model(X_t)
        for idx, i in enumerate(test_output):
            res = torch.argmax(i)
            ohe_res = np.zeros(shape = (1,3))[0]
            ohe_res[res] = 1
            labeled_out = ohe_train.inverse_transform(ohe_res.reshape(1,-1))
            results.append(labeled_out[0][0])    

In [45]:
list(results)

['H',
 'L',
 'L',
 'L',
 'H',
 'H',
 'M',
 'H',
 'H',
 'L',
 'H',
 'H',
 'L',
 'M',
 'L',
 'L',
 'H',
 'M',
 'M',
 'M',
 'L',
 'L',
 'M',
 'H',
 'H',
 'H',
 'H',
 'L',
 'H',
 'L',
 'M',
 'H',
 'H',
 'L',
 'H',
 'H',
 'L',
 'L',
 'M',
 'M',
 'M',
 'L',
 'M',
 'H',
 'L',
 'H',
 'H',
 'M',
 'L',
 'M',
 'M',
 'H',
 'L',
 'M',
 'H',
 'M',
 'L',
 'H',
 'M',
 'L',
 'L',
 'H',
 'M',
 'L',
 'H',
 'M',
 'M',
 'M',
 'L',
 'H',
 'L',
 'L']

In [46]:
list(y_test_org)

['M',
 'L',
 'M',
 'M',
 'H',
 'H',
 'M',
 'H',
 'H',
 'L',
 'M',
 'M',
 'L',
 'M',
 'L',
 'L',
 'H',
 'M',
 'M',
 'H',
 'L',
 'M',
 'M',
 'M',
 'M',
 'H',
 'H',
 'L',
 'M',
 'M',
 'H',
 'H',
 'H',
 'L',
 'M',
 'H',
 'L',
 'L',
 'M',
 'M',
 'H',
 'L',
 'M',
 'H',
 'L',
 'H',
 'H',
 'M',
 'M',
 'M',
 'M',
 'H',
 'M',
 'M',
 'H',
 'M',
 'L',
 'H',
 'M',
 'L',
 'L',
 'M',
 'M',
 'M',
 'H',
 'L',
 'M',
 'M',
 'L',
 'H',
 'L',
 'L']

In [47]:
# sklearn libraries
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import LocalOutlierFactor
from sklearn import preprocessing
from sklearn.preprocessing import OneHotEncoder
from sklearn import preprocessing
from sklearn.metrics import classification_report

In [48]:
report = classification_report(list(y_test_org),results,output_dict=True)
report_df = pd.DataFrame(report).transpose()
report_df

,precision,recall,f1-score,support
H,0.692308,0.857143,0.765957,21.000000
L,0.720000,0.947368,0.818182,19.000000
M,0.809524,0.531250,0.641509,32.000000
accuracy,0.736111,0.736111,0.736111,0.736111
macro avg,0.740611,0.778587,0.741883,72.000000
weighted avg,0.751711,0.736111,0.724429,72.000000
